In [1]:
import os
os.environ['OPENAI_API_KEY'] = ''
api_key=''
import os
os.environ['GROQ_API_KEY'] = api_key

In [3]:
# Optional: add tracing to visualize the agent trajectories
import os
from getpass import getpass

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [4]:
import operator
from typing import Annotated, Any, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
import json 
class State(TypedDict):
    input: str
    responses: str
    feedback: str
    aggregated_response: Annotated[list, operator.add]
    final_answer: str
    iteration: int
    max_iterations: int
    reflection: str

# Initialize models and search tool
proposer_models = [
    ChatGroq(temperature=0, model_name="llama3-70b-8192",),
    ChatGroq(temperature=0, model_name="llama3-8b-8192",),
    ChatGroq(temperature=0, model_name="mixtral-8x7b-32768"),
]

aggregator_model = ChatOpenAI(model="gpt-4o", temperature=0)
reflector_model = ChatOpenAI(model="gpt-4o", temperature=0)



/Users/pradeep.borado/.conda/envs/cv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:


class Proposer:
    def __init__(self, model: ChatOpenAI, id: str):
        self.model = model
        self.id = id

    def __call__(self, state: State) -> Dict[str, Dict[str, str]]:
        
        prompt = ChatPromptTemplate.from_messages([
            ("human", "Please provide a response to the following prompt: {input}\n\n"
             "The previous response was {response}\n\n"
             "The provided feedbackw was {feedback} \n\n"
                      "Always answer with as much detail as possible.")
        ])
        response = self.model(prompt.format_messages(
            input=state["input"],
            response=state['responses'],
            feedback=state['feedback']
            
        ))
        
        
        
        return {"aggregated_response": [json.dumps({self.id: response.content})]}
        



In [ ]:
def aggregator(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        #("human", "Given the following responses to the prompt '{input}', please synthesize them into a single, high-quality response:\n\n{responses}")
        ("human", '''
        You have been provided with a set of responses from various open-source models to the  user query '{input}'.
        Your task is to synthesize these responses into a single, high-quality response.
        It is crucial to critically evaluate the information provided in these responses, 
        recognizing that some of it may be biased or incorrect. 
        Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.
 
        Responses from models:
        \n\n{responses}

''')
    ])
    response = aggregator_model(prompt.format_messages(
        input=state["input"],
        responses="\n\n".join(state["aggregated_response"])
    ))
    state["responses"] = response.content
    return state

In [5]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser


class GradeGeneration(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    score: str = Field(
        description="Is this the correct answer to the question, 'yes' or 'no'"
    )
    feedback: str = Field(
        description="Provided specific feedback for improvement"
    )
        
def reflector(state: State) -> State:
    parser = PydanticOutputParser(pydantic_object=GradeGeneration)

    prompt  = ChatPromptTemplate.from_messages([
            ("human", "Given the following answer to the question: '{input}'\n\n"
            "Answer: {aggregated_response}\n\n"
                      "If the answer is satisfactory and complete, grade it as yes. \n"
    "Provide json object with  feedback string and binary score 'yes' or 'no' score to indicate whether the answer is correct"
            )
        ])
    chain = prompt | reflector_model | parser

    response = chain.invoke({
        'input':state["input"],
        'aggregated_response':state["responses"]
    })
    state["reflection"] = response.score
    state['feedback'] = response.feedback
    state["iteration"] += 1
    return state

In [6]:

def should_continue(state: State) -> str:
    if state["iteration"] >= state["max_iterations"]:# or "YES" in state["reflection"].upper():
        state["final_answer"] = state["responses"]
        return "end"
    return "refine"

# Create the graph
workflow = StateGraph(State)

# Add nodes
for i, model in enumerate(proposer_models):
    workflow.add_node(f"proposer_{i}", Proposer(model, f"proposer_{i}"))
workflow.add_node("aggregator", aggregator)
workflow.add_node("reflector", reflector)

# Define edges
workflow.set_entry_point("proposer_0")
for i in range(1, len(proposer_models)):
    workflow.add_edge(f"proposer_0", f"proposer_{i}")
    workflow.add_edge(f"proposer_{i}", "aggregator")
workflow.add_edge("aggregator", "reflector")
workflow.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "refine": "proposer_0",
        "end": END
    }
)

In [ ]:
app = workflow.compile()

In [8]:
def query_moa(question):
    # Run the graph
    initial_state = {
        "input": question,
        "responses": '',
        'feedback':"",
        "aggregated_response": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3,
        "reflection": ""
    }
    for output in app.stream(initial_state):
        for key, value in output.items():
            print(f"Finished running: {key}:")
    print(value['responses'])
    return value

In [17]:
value = query_moa("Explain the latest advancements in quantum computing and their potential applications.")

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
### Latest Advancements in Quantum Computing and Their Potential Applications

Quantum computing has made remarkable strides in recent years, with significant advancements in hardware, software, and algorithm development. These innovations are pushing the boundaries of what is possible with this revolutionary technology, promising to transform various industries and fields.

#### Latest Advancements in Quantum Computing

1. **Quantum Processors with Higher Qubit Counts**:
   - Companies like Google, IBM, and Rigetti Computing have developed quantu

In [20]:
value=query_moa("Write the snake game in python without any errors")

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Here is a comprehensive and refined implementation of the classic Snake game in Python using the Pygame library. This version includes detailed explanations and improvements to ensure clarity and functionality.

```python
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up some constants
WIDTH, HEIGHT = 800, 600
BLOCK_SIZE = 20
SPEED = 15

# Set up some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (50, 153, 213)
YELLOW = (255, 255, 102)

# Set up the display
screen = 

In [10]:
value = query_moa("if we lay 5 shirts in the sun and it takes 4 hours to dry, how long would 20 shirts take to dry?")

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
To determine how long it would take for 20 shirts to dry in the sun, we need to consider the relationship between the number of shirts and the drying time. Initially, it might seem logical to assume that the drying time is directly proportional to the number of shirts. However, this assumption oversimplifies the complex factors that influence the drying process.

### Key Factors Affecting Drying Time:

1. **Exposure to Sunlight**: The amount of direct sunlight each shirt receives is crucial. If the shirts are laid out in a way that maximizes expos

In [11]:
value = query_moa('''Maria is staying at a hotel that charges $99.95 per night plus tax for a room. A tax of 8% is applied to the room rate, 
and an additional onetime untaxed fee of $5.00 is charged by the hotel. 
Which of the following represents Maria’s total charge, in dollars, for staying x nights?  (99.95 + 0.08x) + 5  1.08(99.95x) + 5  1.08(99.95x + 5)  1.08(99.95 + 5)x''')

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
To determine Maria's total charge for staying \( x \) nights at a hotel, we need to consider the following components:

1. **Nightly Room Rate**: $99.95 per night.
2. **Tax**: 8% of the nightly room rate.
3. **One-time Fee**: $5.00, which is not subject to tax.

### Step-by-Step Calculation

1. **Calculate the Tax on the Nightly Room Rate**:
   - The tax is 8% of $99.95.
   - Tax per night = 0.08 × 99.95 = 7.996.

2. **Total Cost per Night Including Tax**:
   - Total cost per night = Room rate + Tax
   - Total cost per night = 99.95 + 7.996 = 107.

In [12]:
value =  query_moa('''There are three killers in a room.
Someone enters the room and kills one of them. Nobody leaves the room. How many killers are left in the room? Explain your reasoning step by step.''')

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
To determine how many killers are left in the room after the described events, let's break down the situation step by step:

### Initial Situation
- There are three killers in the room. This means that each of these individuals has committed a murder or is capable of doing so.

### Event
- Someone enters the room and kills one of the killers. This event is crucial in understanding the subsequent situation. The person who enters the room is not initially considered a killer, but their action changes their status.

### New Situation
- Nobody leaves 

In [13]:
value =  query_moa('''A marble is put in a glass.
The glass is then turned upside down and put on a table.
Then the glass is picked up and put in a microwave. 
Where's the marble? Explain your reasoning step by step.''')

Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
To determine the final location of the marble, let's break down the sequence of events step by step, considering the physical behavior of the marble and the glass:

### Step 1: A marble is put in a glass.
At this point, the marble is inside the glass, resting at the bottom due to gravity. The glass is upright, and the marble is stable within the glass, surrounded by the glass walls and floor.

### Step 2: The glass is then turned upside down and put on a table.
When the glass is turned upside down, gravity will cause the marble to move towards the

In [15]:
value = query_moa('Give me 10 sentences that end in the word Apple.')

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Here are 10 well-crafted sentences that end with the word "Apple," synthesized from the provided responses:

1. The crunchy snack I love to eat is a juicy Apple.
2. The teacher asked the student to draw a picture of a shiny Apple.
3. The farmer carefully plucked the ripest fruit from the tree, a perfect Apple.
4. The tech giant's latest product is the iPhone, made by Apple.
5. The healthy breakfast option I chose was oatmeal with a slice of Apple.
6. The autumn festival featured a contest to see who could eat a pie filled with Apple.
7. The sweet 

In [16]:
value = query_moa('''It takes one person 5 hours to dig a 10 foot hole in the ground. 
How long would it take 50 people to dig a single 10 foot hole?'

''')

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
To determine how long it would take 50 people to dig a single 10-foot hole, we need to consider both theoretical calculations and practical limitations.

### Theoretical Calculation

1. **Individual Rate of Work**:
   - One person can dig a 10-foot hole in 5 hours.
   - Therefore, the rate of digging is \( \frac{10 \text{ feet}}{5 \text{ hours}} = 2 \text{ feet per hour} \).

2. **Combined Rate of 50 People**:
   - If 50 people work together, their combined rate would be \( 50 \times 2 \text{ feet per hour} = 100 \text{ feet per hour} \).

3. **Ti